In [1]:
import logging
import logging.config
import os
import json
import ConfigParser
from simpleMenu import menu

from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError


# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [2]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [54]:
class configuration(object):
    def __init__(self, configpath='./', configfile='config.ini'):
        self.logger = logging.getLogger(__name__)
        self.configpath = os.path.expandvars(configpath)
        self.configfile = os.path.join(self.configpath, configfile)
        self.getConfig()
        
    def writeConfig(self):
        if not os.path.exists(self.configpath):
            try:
                logging.warn('configuration path does not exist; creating: {}'.format(self.configpath))
                os.makedirs(self.configpath)
            except Exception as e:
                logging.error(e)
            

        
    def getConfig(self, mainsection = 'Main'):
        self.parser = ConfigParser.SafeConfigParser()
        #credentialspath = os.path.join(self.configpath, 'credentials')
      

        requiredprefs = {
                            'credentials': [self.parser.get, os.path.join(self.configpath, 'credentials')],
            
                        }
        
        # check if config path exists
        if len(self.parser.read(self.configfile)) <= 0:
            self.logger.warn('no configuration file found at: {}'.format(self.configfile))
            writeconfig = True
        
        if not self.parser.has_section(mainsection):
            self.parser.add_section(mainsection)
            
        preferences = {}
        for key in requiredprefs:
            try:
                preferences[key] = requiredprefs[key][0](mainsection, key)
            except (ConfigParser.NoSectionError, ConfigParser.NoOptionError):
                self.parser.set(mainsection, key, requiredprefs[key][1])
                preferences[key] = requiredprefs[key][1]
                
        for key in requiredprefs:
            exec ("self.{0} = preferences['{0}']".format(key))
        
        if writeconfig:
            self.writeConfig()
            
        
        
    def printConfig(self):
        for section in self.parser.sections():
            print 'section: {0}'.format(section)
            for key in self.parser.options(section):
                print '{0} = {1}'.format(key, self.parser.get(section, key))
            

                
    

In [55]:
myconfig = configuration(configpath='/Users/aaronciuffo/.config/'+appName)


2018-08-13 17:18:56,729: [WARNING: __main__.getConfig] no configuration file found at: /Users/aaronciuffo/.config/portfolio_creator/config.ini
2018-08-13 17:18:56,730: [WARNING: root.writeConfig] configuration path does not exist; creating: /Users/aaronciuffo/.config/portfolio_creator


In [56]:
myconfig.printConfig()

section: Main
credentials = /Users/aaronciuffo/.config/portfolio_creator/credentials


In [21]:

version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.cfg'
cfgpath = os.path.join('~/.config/', appName)

logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

# google drive authorization
logging.info('checking google credentials')
try:
    credentials = getCredentials(storage_path=config.credentialstore)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

# print '\n'*10
# print 'Welcome to the portfolio creator for Google Team Drive version {0}'.format(version)
# print 'This program will create portfolio folders in Google Team Drive for students.'
# print 'You will need a student_export.text file from PowerSchool with at least the following information:'
# print '     ClassOf, FirstLast, Student_Number\n'
# print 'The order of the CSV does not matter, but the headers must be on the very first line.'
# print 'You will also need Google File Stream installed and configured'
# print 'File Stream can be downloaded here: https://support.google.com/drive/answer/7329379?hl=en'
# raw_input("Press Enter to continue...\n\n\n")



2018-08-13 16:48:10,501: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-08-13 16:48:10,502: [INFO: root.<module>] checking google credentials
2018-08-13 16:48:10,503: [DEBUG: gdrive.auth.getCredentials] preparing google drive credentials
2018-08-13 16:48:10,504: [DEBUG: root.getCredentials] checking for credential store directory: ~/.config/portfolio_creator/credentials/credentials
2018-08-13 16:48:10,506: [DEBUG: root.getCredentials] credential store accepted
2018-08-13 16:48:10,507: [INFO: googleapiclient.discovery._retrieve_discovery_doc] URL being requested: GET https://www.googleapis.com/discovery/v1/apis/drive/v3/rest
